# Stream Ciphers

treats the plaintext as a stream and encrypts the bits individually

smaller and faster 

assumed to be more efficient

encryption and decryption are the same procedure -XOR

they can reach very high level of secrecy

One time pod: key as long as the plaintext, uniformly distributed in the key space, key must be used only once -> information theory-wise secure

Security in stream ciphers relies on key stream

OTP is unpractical because stream ciphers rely on random number generators

-maybe talk a bit about RNGs- 

RNG must be reproducible and unpredictable



In [1]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

In [2]:
random = [1, 0, 1, 0, 1, 0, 1, 0]
bits = Bits(random)
bits

Bits([True, False, True, False, True, False, True, False])

## LFSR

one of the main building blocks of PRNGs

In [3]:
lfsr = LFSR({1, 2, 5}, 0b000)
bits = lfsr.cycle()
print(bits)

0


In [4]:
lfsr = LFSR({1, 2, 5}, 0b00101)
output = lfsr.run_steps(10)

In [5]:
lfsr.__next__()

False

In [6]:
output

Bits([False, True, False, False, True, True, True, False, True, False])

In [7]:
bits = lfsr.cycle(state=0b00101)
bits

Bits([False, True, False, False, True, True, True])

## Berlekamp-Massey Algorithm

binary output sequence -> shortest lfsr able to produce the sequence

exploits the ptoperty that x p_i+b[t-i] must be zero

makes the system prone to KPA attck. If eve knows enough x_i and y_i's she can compute b_i's and apply B-M algorithm to infer P(X)

In [8]:
test_bits = Bits([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1])
print(berlekamp_massey(test_bits))


{0, 3, 5}


In [9]:
with open('binary_sequence.bin', 'rb') as f:
        binary_sequence = f.read()

In [10]:
binary_sequence[:50]

b'\xbb`\xef\x067\xae\xd0K"Vd]#Q\xeb\x02~<\xe6C\xbe\xed5\xd0\xec\xada\xe8\x89h\xf3\xbdFc\x96\xb5\x8e\xb0\x03\xabVFY#\xd1\xeb">\xb5\xe4'

In [11]:
bits = Bits(binary_sequence)

In [12]:
poly = berlekamp_massey(bits)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 18, 7}
Linear complexity: 18


## Alternating Step Generator

In [13]:
alt_step = AlternatingStep()

In [14]:
bits = alt_step.run(25)

In [15]:
bits

Bits([False, False, False, True, True, True, False, True, False, True, False, True, True, True, False, False, False, True, False, False, True, False, True, True, False])

# KPA

In [16]:
with open("ciphertext.bin", "rb") as f:
    ciphertext = f.read()

with open("known-plaintext.txt", "r", encoding="utf-8") as f:
    known_plaintext = f.read().encode("utf-8")

In [17]:
print(f"Ciphertext: {ciphertext[:30]} ..., \nKnown-Plaintext: {known_plaintext[:90]}...")

Ciphertext: b"\xb7;\xcep\x9e\x7f\xc0\xe3H_'\xc6D\x9b\xe8\xbd\x8e[\x8b\xb0\x94\x00\xdf]\xa9\xd9\x152k\x06" ..., 
Known-Plaintext: b'The Legacy of the Hidden Key\n\nIn a quiet corner of the university library, where dust mote'...


In [18]:
kp_bits = Bits(known_plaintext)
length = len(kp_bits)
cipher_bits = Bits(Bits(ciphertext)[:length])

In [19]:
bit_sequence = kp_bits ^ cipher_bits
print(f"Bit sequence: {bit_sequence[:10]} ...")

Bit sequence: [True, True, True, False, False, False, True, True, False, True] ...


In [20]:
poly = berlekamp_massey(bit_sequence)
linear_complexity = max(poly) if poly else 0

print("Shortest feedback polynomial degrees:", poly)
print("Linear complexity:", linear_complexity)

Shortest feedback polynomial degrees: {0, 1, 9, 48, 19}
Linear complexity: 48


In [22]:
import numpy as np
from typing import Set

def build_lfsr_matrix(poly_degrees: Set[int], num_rows: int) -> np.ndarray:
    """
    Build the LFSR matrix over GF(2) using the feedback polynomial.
    Each row represents a linear equation of how one output depends on the previous bits.
    """
    L = max(poly_degrees)
    poly = [0] * L
    for deg in poly_degrees:
        if deg != 0:  # skip the constant term
            poly[L - deg] = 1  # reverse indexing to align with state bits

    M = np.zeros((num_rows, L), dtype=np.uint8)
    for i in range(num_rows):
        if i < L:
            M[i, i] = 1
        else:
            M[i] = np.zeros(L, dtype=np.uint8)
            for j in range(1, L + 1):
                if poly[j - 1] == 1:
                    M[i] ^= M[i - j]
    return M

# Step 1: Take 96 known output bits (we need at least 48 to solve a 48-bit state)
L = max(poly)
known_bits = kp_bits[:L * 2]  # 96 bits

# Step 2: Build matrix M and target vector
M = build_lfsr_matrix(poly, len(known_bits))
output_vec = np.array(known_bits, dtype=np.uint8)

# Step 3: Solve M[:L] * state = output[:L] over GF(2)
from scipy.linalg import lu

def gf2_solve(M, b):
    """ Solve Mx = b over GF(2) using Gaussian elimination """
    A = M[:L].copy()
    b = b[:L].copy()
    n = len(b)

    # Forward elimination
    for col in range(n):
        if A[col, col] == 0:
            for row in range(col + 1, n):
                if A[row, col] == 1:
                    A[[col, row]] = A[[row, col]]
                    b[[col, row]] = b[[row, col]]
                    break
        for row in range(col + 1, n):
            if A[row, col] == 1:
                A[row] ^= A[col]
                b[row] ^= b[col]

    # Back substitution
    x = np.zeros(n, dtype=np.uint8)
    for row in range(n - 1, -1, -1):
        x[row] = b[row]
        for col in range(row + 1, n):
            x[row] ^= A[row, col] & x[col]

    return x

# Step 4: Recover the initial state
initial_state_vec = gf2_solve(M, output_vec)
initial_state = Bits(initial_state_vec.tolist())
initial_state.bits[:L]  # Show recovered state bits

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True]

In [23]:
lfsr = LFSR(poly, initial_state)
bits = lfsr.run_steps(len(cipher_bits))

In [24]:
decrypted_bits = bits ^ cipher_bits
decrypted_bits[:10]

[True, True, True, True, True, False, True, True, False, True]

In [25]:
def bits_to_bytes(bits):
    """Convert a Bits object to bytes."""
    byte_array = bytearray()
    for i in range(0, len(bits), 8):
        byte = 0
        for bit in bits[i:i+8]:
            byte = (byte << 1) | bit
        byte_array.append(byte)
    return bytes(byte_array)

# Convert decrypted bits to bytes
decrypted_bytes = bits_to_bytes(decrypted_bits)

# Decode bytes to text
decrypted_text = decrypted_bytes.decode("utf-8", errors="replace")
print(decrypted_text[:1000])  # Preview the first 1000 characters

�_�<�+}�C) �������1�$:X~�\*��>��$��q�V�Z �F-�4��09`�KI3Y���
^d}��r�3�W�}a�*�'?� ;��c�����E?��m\�7<ӚZ�v��C>���Wԓ41����m��^����k����_��[Ke���S�����ldM�l wSH<�:v��=-
���z)�@;=�Y��]	,!a�Z��r�ӹ�7��@"�2sy�EV'��z4!r]��To:=/Lm@��=w&~ȇ��<�%�W�J=�uMܻ�9��T��=N�Tŧ�tĀt���~K潡ր���Lȧѫ7|l�S�;���> �&���s�_!
�Q��t�����|BtW�BV��P�����j�%}�&n�t�d����c���7j��1%�k��m~��MЪ�5�y���p���k-��Y�'�(������L�c:�		�)Q��]�t
